In [40]:
!sudo apt update

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 2s (169 kB/s)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
15 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [2]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [3]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# !cp /content/drive/MyDrive/spark-3.3.1-bin-hadoop3.tgz /content/

In [ ]:
#!wget https://archive.apache.org/dist/spark/spark-3.1.1/spark-3.3.1-bin-hadoop3.tgz

In [ ]:
!tar -zxvf spark-3.3.1-bin-hadoop3.tgz

In [8]:
!pip install -q findspark

In [ ]:
!pip install pyspark

In [10]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [11]:
import findspark
findspark.init()
findspark.find()

'/content/spark-3.3.1-bin-hadoop3'

In [12]:
import pyspark

In [74]:
sc= pyspark.SparkContext()

from pyspark.sql import DataFrame,SparkSession

spark = SparkSession.builder.appName("IoTDataAnalysis").getOrCreate()

In [75]:
spark

In [50]:
df = spark.read.format("json") \
                         .option("header", "true") \
                         .option("inferSchema", "true") \
                         .load("/content/drive/MyDrive/iot_devices.json")

In [51]:
df.printSchema()

root
 |-- battery_level: long (nullable = true)
 |-- c02_level: long (nullable = true)
 |-- cca2: string (nullable = true)
 |-- cca3: string (nullable = true)
 |-- cn: string (nullable = true)
 |-- device_id: long (nullable = true)
 |-- device_name: string (nullable = true)
 |-- humidity: long (nullable = true)
 |-- ip: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- lcd: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- scale: string (nullable = true)
 |-- temp: long (nullable = true)
 |-- timestamp: long (nullable = true)



In [52]:
print(df)

DataFrame[battery_level: bigint, c02_level: bigint, cca2: string, cca3: string, cn: string, device_id: bigint, device_name: string, humidity: bigint, ip: string, latitude: double, lcd: string, longitude: double, scale: string, temp: bigint, timestamp: bigint]


In [53]:
df.show()

+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|           cn|device_id|         device_name|humidity|             ip|latitude|   lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|            8|      868|  US| USA|United States|        1|meter-gauge-1xbYRYcj|      51|   68.161.225.1|    38.0| green|    -97.0|Celsius|  34|1458444054093|
|            7|     1473|  NO| NOR|       Norway|        2|   sensor-pad-2n2Pea|      70|  213.161.254.1|   62.47|   red|     6.15|Celsius|  11|1458444054119|
|            2|     1556|  IT| ITA|        Italy|        3| device-mac-36TWSKiT|      44|      88.36.5.1|   42.83|   red|    12.83|Celsius|  19|1458444054120|
|            6|     1080|  US| USA|United Stat

In [56]:
from pyspark.sql.functions import col, column
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [57]:
df.createOrReplaceTempView("iot")

In [63]:
device_count_by_country = spark.sql("SELECT cca3 AS Country, COUNT(*) AS DeviceCount FROM iot GROUP BY cca3 ORDER BY DeviceCount DESC")
device_count_by_country.show()

+-------+-----------+
|Country|DeviceCount|
+-------+-----------+
|    USA|      70405|
|    CHN|      14455|
|    JPN|      12100|
|    KOR|      11879|
|    DEU|       7942|
|    GBR|       6486|
|    CAN|       6041|
|    RUS|       5989|
|    FRA|       5305|
|    BRA|       3224|
|    AUS|       3119|
|    ITA|       2915|
|    SWE|       2880|
|    POL|       2744|
|    NLD|       2488|
|    ESP|       2310|
|    TWN|       2128|
|    IND|       1867|
|    CZE|       1507|
|    NOR|       1487|
+-------+-----------+
only showing top 20 rows



In [66]:
high_co2_countries = spark.sql("SELECT cca3 AS Country FROM iot WHERE C02_level > 1400 ORDER BY C02_level DESC")
high_co2_countries.show()

+-------+
|Country|
+-------+
|    BGR|
|    USA|
|    EGY|
|    TWN|
|    DEU|
|    DEU|
|    CAN|
|    JPN|
|    DEU|
|    BGR|
|    ARG|
|    GBR|
|    USA|
|    USA|
|    USA|
|    CHN|
|    KOR|
|    USA|
|    NOR|
|    POL|
+-------+
only showing top 20 rows



In [65]:
devices_high_co2 = spark.sql("SELECT cca3 as Country,device_name,device_id FROM iot WHERE lcd = 'red' GROUP BY cca3,device_name,device_id ORDER BY device_id")
devices_high_co2.show()

+-------+--------------------+---------+
|Country|         device_name|device_id|
+-------+--------------------+---------+
|    NOR|   sensor-pad-2n2Pea|        2|
|    ITA| device-mac-36TWSKiT|        3|
|    JPN|sensor-pad-8xUD6p...|        8|
|    USA|sensor-pad-10Bsyw...|       10|
|    ITA|meter-gauge-11dlM...|       11|
|    USA|sensor-pad-16aXmI...|       16|
|    USA|meter-gauge-17zb8...|       17|
|    USA|meter-gauge-19eg1...|       19|
|    JPN|  sensor-pad-22oWV2D|       22|
|    CAN|sensor-pad-24Pytz...|       24|
|    KOR|  device-mac-27P5wf2|       27|
|    KOR|sensor-pad-28Tsud...|       28|
|    UKR|meter-gauge-47WsF9s8|       47|
|    SWE|meter-gauge-534fD...|       53|
|    USA|sensor-pad-5410CW...|       54|
|    USA|device-mac-57tRB0...|       57|
|    USA|  sensor-pad-64djcIn|       64|
|    CZE|sensor-pad-663dzo...|       66|
|    IND|meter-gauge-77IKW...|       77|
|    KOR|sensor-pad-78PkeI...|       78|
+-------+--------------------+---------+
only showing top

In [67]:
devices_low_battery = spark.sql("SELECT device_id, cca3 AS Country FROM iot WHERE battery_level < 20")
devices_low_battery.show()

+---------+-------+
|device_id|Country|
+---------+-------+
|        1|    USA|
|        2|    NOR|
|        3|    ITA|
|        4|    USA|
|        5|    PHL|
|        6|    USA|
|        7|    CHN|
|        8|    JPN|
|        9|    JPN|
|       10|    USA|
|       11|    ITA|
|       12|    USA|
|       13|    IND|
|       14|    NOR|
|       15|    USA|
|       16|    USA|
|       17|    USA|
|       18|    CHN|
|       19|    USA|
|       20|    USA|
+---------+-------+
only showing top 20 rows



In [72]:
sc.stop()

In [73]:
spark.stop()